In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('./data/training-data-with-entities.csv')

In [3]:
df.head(3)

,Intent,Utterance,Entitiy,Id
0,PlayRadioIntent,start playing radio w. m. ninety five point six,radio w. m. ninety five point six,bbc_wm
1,PlayRadioIntent,start radio shropshire please,radio shropshire,bbc_radio_shropshire
2,PlayRadioIntent,let me listen to b. b. c. london please,b. b. c. london,bbc_london


## How many PlayPodcastIntent Utterances contain  the word 'podcast'?

In [4]:
seriesPodcastIntentsWithPodcast = df.apply(lambda x: True if x['Intent'] == 'PlayPodcastIntent' and 'podcast' in x['Utterance'] else False, axis=1)

numOfRows = len(seriesPodcastIntentsWithPodcast[seriesPodcastIntentsWithPodcast == True].index)
 
print('Number of PlayPodcastIntent Utterances that feature \'podcast\' at all : ', numOfRows)


Number of PlayPodcastIntent Utterances that feature 'podcast' at all :  1072070


## How many PlayRadioIntent Utterances contain  the word 'radio'?

In [5]:
seriesRadioIntentsWithRadio = df.apply(lambda x: True if x['Intent'] == 'PlayRadioIntent' and 'radio' in x['Utterance'] else False, axis=1)
 
numOfRows = len(seriesRadioIntentsWithRadio[seriesRadioIntentsWithRadio == True].index)
 
print('Number of PlayRadioIntent Utterances that feature \'radio\' at all : ', numOfRows)


Number of PlayRadioIntent Utterances that feature 'radio' at all :  9842


## How many PlayPodcastIntent Utterances contain  the word 'radio'?

In [6]:
seriesPodcastIntentsWithRadio = df.apply(lambda x: True if x['Intent'] == 'PlayPodcastIntent' and 'radio' in x['Utterance'] else False, axis=1)

numOfRows = len(seriesPodcastIntentsWithRadio[seriesPodcastIntentsWithRadio == True].index)
 
print('Number of PlayPodcastIntent Utterances that feature \'radio\' at all : ', numOfRows)


Number of PlayPodcastIntent Utterances that feature 'radio' at all :  1066674


## How many PlayRadioIntent Utterances contain  the word 'podast'?

In [7]:
seriesRadioIntentsWithPodcast = df.apply(lambda x: True if x['Intent'] == 'PlayRadioIntent' and 'podcast' in x['Utterance'] else False, axis=1)

numOfRows = len(seriesRadioIntentsWithPodcast[seriesRadioIntentsWithPodcast == True].index)
 
print('Number of PlayPodcastIntent Utterances that feature \'podcast\' at all : ', numOfRows)

Number of PlayPodcastIntent Utterances that feature 'podcast' at all :  0


If the utterance contains the word 'podcast' then it is extremely unlikely that the user is asking for a radio station. This could be useful.

This means that a manual rule could be applied so that that if the utterance contains the word 'podcast' then a it will automatically be mapped to `PlayPodcastIntent` and would not need to go through any further prediction algorithm.

In [8]:
df.tail()

,Intent,Utterance,Entitiy,Id
1976576,PlayPodcastIntent,start podcast five live science podcast bbc ra...,five live science podcast bbc radio five live,p02pc9ny
1976577,PlayPodcastIntent,let me hear bbc newcastle rebecca o'neill,bbc newcastle rebecca o'neill,p06nzv9h
1976578,PlayPodcastIntent,continue listening to bbc world service world ...,bbc world service world update,p007dhp8
1976579,PlayPodcastIntent,start playing the five faces of leonardo please,the five faces of leonardo,m0004l8g
1976580,PlayPodcastIntent,can I hear podcast b. b. c. radio cumbria joe ...,b. b. c. radio cumbria joe costin,p001d72n


## How Would the Model Perform on the other Radio Stations?

In [9]:
radio_intent_df = df[~(df.Intent != 'PlayRadioIntent')].copy()

In [11]:
radio_intent_df.shape

(16761, 4)

### Import the Model 

In [12]:
import pickle
model = pickle.load(open('trained_model.pickle', 'rb'))

A quick testing to ensure that the model has imported correctly

In [13]:
model.predict(['the cat sat on the radio', 'play b. b. c. radio two please'])

array(['PlayPodcastIntent', 'PlayRadioIntent'], dtype=object)

## Using the Model to Classify alongside the Manual Rule

In [15]:
df['Correct?'] = df.apply(
    lambda x: True if 'podcast' in x['Utterance']
    or model.predict([x['Utterance']]) == x['Intent'] 
    else False, axis=1)


In [26]:
df.head()

,Intent,Utterance,Entitiy,Id,Correct?
0,PlayRadioIntent,start playing radio w. m. ninety five point six,radio w. m. ninety five point six,bbc_wm,True
1,PlayRadioIntent,start radio shropshire please,radio shropshire,bbc_radio_shropshire,True
2,PlayRadioIntent,let me listen to b. b. c. london please,b. b. c. london,bbc_london,True
3,PlayRadioIntent,find me wales radio,wales radio,bbc_radio_wales_fm,True
4,PlayRadioIntent,can I play b. b. c. york please,b. b. c. york,bbc_radio_york,True


In [28]:
df['Correct?'].value_counts()

True     1976153
False        428
Name: Correct?, dtype: int64

The accuracy from just using the model, when tested on the training set was 0.9997369199907922. 

In [31]:
f'The accuracy is now {1 - (428 / 1976153)} which is slightly more accurate.'

'The accuracy is now 0.9997834175795093 which is slightly more accurate.'